In [3]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
# this model have accuracy of 100% and loss of 0.0015
model = load_model('fresh.h5')
folders = ['bottom_left', 'bottom_right', 'bottom_middle', 'top_left', 'top_right', 'top_middle', 'left_middle', 'right_middle','middle']

eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load your pre-trained CNN model
def detect_eyes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
    return eyes


# Define the desired dimensions
desired_width = 250
desired_height = 50


# Initialize the webcam
video_capture = cv2.VideoCapture(0)  # 0 represents the default webcam device
eyes_detected = False

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = video_capture.read()
    frame = cv2.flip(frame, 1)
    eyes = detect_eyes(frame)
    

    if len(eyes) >= 2:
        (eye_1x, eye_1y, eye_1width, eye_1height), (eye_2x, eye_2y, eye_2width, eye_2height) = eyes[:2]
        eye_x = min(eye_1x, eye_2x)
        eye_y = min(eye_1y, eye_2y)
        eye_width = abs(eye_1x - eye_2x) + max(eye_1width, eye_2width)
        eye_height = abs(eye_1y - eye_2y) + max(eye_1height, eye_2height)
        
        eye_roi = frame[eye_y:eye_y+eye_height, eye_x:eye_x+eye_width]

        # Resize the ROI to match the desired dimensions
        resized_roi = cv2.resize(eye_roi, (desired_width, desired_height))

        # Convert the ROI to RGB format
        resized_roi = cv2.cvtColor(resized_roi, cv2.COLOR_BGR2GRAY)

        # Convert the ROI to numpy array
        resized_roi = np.asarray(resized_roi)

        # Expand dimensions to match the input shape of your CNN model
        resized_roi = np.expand_dims(resized_roi, axis=0)

        # Normalize pixel values
        resized_roi = resized_roi / 255.0

        # Pass the preprocessed ROI through your CNN model for prediction
        predictions = model.predict(resized_roi)
        predicted_class = np.argmax(predictions, axis=1)
        # print(predicted_class)
        frame = cv2.rectangle(frame, (eye_x, eye_y), (eye_x+eye_width, eye_y+eye_height), (0, 255, 0), 2)
    #     eyes_detected = True
    # else:
    #     eyes_detected = False
        frame = cv2.putText(frame, folders[predicted_class[0]], (eye_x, eye_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.imshow('Eye Detection', frame)
    

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
video_capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 31ms/step


In [ ]:

# Save the frame in the "eyes_pattern" folder
frame_count += 1
filename = os.path.join(folder_path, f'frame_{frame_count}.jpg')
cv2.imwrite(filename, eye_roi)

In [4]:
# flip all the images in the folder bottom_left
import os
import cv2
list1 = ['bottom_right', 'bottom_middle', 'top_left', 'top_right', 'top_middle', 'left_middle', 'right_middle','middle']
for folder in list1:
    folder_path = folder
    filenames = os.listdir(folder_path)

    for filename in filenames:
        image = cv2.imread(os.path.join(folder_path, filename))
        image = cv2.flip(image, 1)
        cv2.imwrite(os.path.join(folder_path, filename), image)

    
